In [113]:
!pip install webdriver-manager


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [114]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import numpy as np
import pandas as pd
import multiprocessing
import re
from rapidfuzz import process
import unicodedata

# Scraping Linkedin

In [115]:
job_query = "python-developer"
len_jobs = 30

In [116]:
start = time.time()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = f"https://ph.linkedin.com/jobs/{job_query}-jobs"

try:
    driver.get(url)
    ActionChains(driver).send_keys(Keys.ESCAPE).perform()
    job_list = driver.find_elements(By.XPATH, "//ul[@class='jobs-search__results-list']/li")
    
    # job catalog scraping
    jobs_scraped = np.array([])
    for job in job_list:
        descs = []
        try:
            job_title = job.find_element(By.XPATH, ".//h3[@class='base-search-card__title']").get_attribute('innerHTML').strip()
            card = job.find_element(By.XPATH, ".//a[@data-tracking-control-name='public_jobs_jserp-result_search-card']")
            link = card.get_attribute('href')
            descs = [job_title, link]
        except:
            continue
    
        try:
            location = job.find_element(By.XPATH, ".//span[@class='job-search-card__location']").get_attribute('innerHTML')
            location = location.replace('\n', '').strip()
            descs.append(location)
        except:
            descs.append('')
        
        try: 
            company = job.find_element(By.XPATH, ".//h4[@class='base-search-card__subtitle']/a").get_attribute('innerHTML').strip()
            descs.append(company)
        except:
            try:
                company = job.find_element(By.XPATH, ".//h4[@class='base-search-card__subtitle']").get_attribute('innerHTML').strip()
                descs.append(company)
            except:
                descs.append('')
                
        if len(jobs_scraped) == 0:
            jobs_scraped = np.append(jobs_scraped, descs)
        else:
            jobs_scraped = np.vstack([jobs_scraped, descs])

        # limit to max len_jobs only
        if len(jobs_scraped) >= len_jobs:
            break
    
    # if there is only 1 scraped job
    if jobs_scraped.shape == (4,):
        jobs_scraped = np.array([list(jobs_scraped)])
        
    # individual job scraping
    job_descs = np.array([])
    for job in jobs_scraped:
        descs = []
        try:
            # scraping emp_type, job_func, job_desc, posted ago
            driver.get(job[1])
            wait = WebDriverWait(driver, timeout=2)
            desc_job = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//ul[@class='description__job-criteria-list']/li")))
            descs = [job[1]]
            for i in [1,2]:
                try:
                    detail = desc_job[i].find_element(By.XPATH, ".//span")
                    detail = detail.get_attribute('innerHTML').replace('\n', '').strip()
                    descs.append(detail)
                except:
                    descs.append('')
            try:
                desc_gen = driver.find_element(By.XPATH, "//div[@class='description__text description__text--rich']/section/div")
                desc_gen = desc_gen.get_attribute('innerHTML')
                descs.append(desc_gen)
            except:
                descs.append('')
            try:
                posted_ago = driver.find_element(By.XPATH, "//span[@class='posted-time-ago__text topcard__flavor--metadata']")
                posted_ago = posted_ago.get_attribute('innerHTML').replace('\n','').strip()
                descs.append(posted_ago)
            except:
                descs.append('')
        except: 
            descs = [job[3],'','','','']
        if len(job_descs) == 0:
            job_descs = np.append(job_descs, descs)
        else:
            job_descs = np.vstack([job_descs, descs])
        time.sleep(2)
    
    # merging
    jobs_df = pd.DataFrame(jobs_scraped)
    jobs_df.columns = ['title','link', 'location', 'company']
    job_descs_df = pd.DataFrame(job_descs)
    
    # if there is only 1 scraped job
    if job_descs_df.shape == (5,1):
        job_descs_df = job_descs_df.T
    
    job_descs_df.columns = ['link','emp_type', 'job_func', 'job_desc', 'posted']
    linkedin_df = jobs_df.merge(job_descs_df, on='link', how='left')
except:
    print("Unable to Scrape Linkedin")
finally:
    # close driver
    try:
        driver.close()
    except:
        print("Unable to Scrape Linkedin")
    
end = time.time()
print("TOTAL TIME", end-start)

TOTAL TIME 119.15519165992737


# Scraping FoundIt

In [117]:
job_query = "python-developer"
len_jobs = 30

In [118]:
start = time.time()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = f"https://www.foundit.com.ph/search/{job_query}-jobs"

try:
    jobs_scraped = np.array([])
    page_num = 1
    while True:
        driver.get(f"{url}{f"-{page_num}" if page_num > 1 else ''}")
        job_list = driver.find_elements(By.XPATH, "//div[@class='srpResultCard']/div")
        if len(job_list) == 0:
            break
        job_list = job_list[1:] # remove the header
        for job in job_list:
            # scraping title, company, url
            try:
                job_title = job.find_element(By.XPATH, ".//a[@title]")
                link = job_title.get_attribute('href')
                job_title = job_title.get_attribute('innerHTML').replace('\n', '').strip()
            except:
                continue
            try:
                company = job.find_element(By.XPATH, ".//div[@class='companyName']/span")
                company = company.get_attribute('innerHTML').replace('\n', '').strip()
            except: 
                company = ''
            descs = [job_title, company, link]
        
            # scraping location, posted ago
            job.find_element(By.XPATH, './/div[@onclick]/div').click()
            wait = WebDriverWait(driver, 5)
            try:
                desc_job = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@id='jobHighlight']/div/div/div")))
                for i in [0,2]:
                    if i != 2:
                        detail = desc_job[i].find_element(By.XPATH, ".//div[@class='details']")
                        detail = detail.get_attribute('innerHTML').replace('\n', '').strip()
                    else: 
                        detail = desc_job[i].find_element(By.XPATH, ".//span[@class='btnHighighlights']")
                        detail = detail.get_attribute('innerHTML').replace('\n', '').split('</i>')
                        detail = detail[1].strip()
                    descs.append(detail)
            except:
                descs.append('','','')
        
            # scraping emp_type, job function, general job_desc
            try:
                desc_job_2 = wait.until(EC.visibility_of_all_elements_located((By.XPATH, ".//div[@id='jobDetail']/div/div")))
                for i in [0,2]:
                    try:
                        detail = desc_job_2[i].find_element(By.XPATH, ".//div[@class='jobDesc']")
                        detail = detail.get_attribute('innerHTML').replace('\n', '').strip()
                        descs.append(detail)
                    except:
                        descs.append('')
            except:
                descs.extend(['',''])
            try:
                desc_gen = wait.until(EC.visibility_of_element_located((By.XPATH, ".//p[@class='jobDescInfo']")))
                desc_gen = desc_gen.get_attribute('innerHTML')
                descs.append(desc_gen)
            except:
                descs.append('')
                
            if len(jobs_scraped) == 0:
                jobs_scraped = np.append(jobs_scraped, descs)
            else:
                jobs_scraped = np.vstack([jobs_scraped, descs])

        # limit to max len_jobs only
        if len(jobs_scraped) >= len_jobs:
            break
        page_num += 1

    # converting to DataFrame
    if jobs_scraped.shape == (8,):
        jobs_scraped = np.array([list(jobs_scraped)])
    
    foundit_df = pd.DataFrame(jobs_scraped)
    foundit_df.columns = ['title','company','link','location','posted','emp_type','job_func','job_desc']
except:
    print("Unable to Scrape Foundit")
finally:
    # close driver
    try:
        driver.close()
    except:
        print("Unable to Scrape Foundit")
    
end = time.time()
print("TOTAL TIME", end-start)

TOTAL TIME 29.090785026550293


# Scraping Jobstreet

In [119]:
job_query = "python-developer"
len_jobs = 30

In [120]:
start = time.time()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = f"https://ph.jobstreet.com/{job_query}-jobs"

try:
    driver.get(url)
    
    # job catalog scraping
    jobs_scraped = np.array([])
    wait = WebDriverWait(driver, 5)
    while True:
        try:
            job_list = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//article[@data-automation='normalJob']")))
            for job in job_list:
                descs = []
                # scraping title, company, link
                try:
                    job_title = job.find_element(By.XPATH, ".//a[@data-automation='jobTitle']")
                    link = job_title.get_attribute('href')
                    job_title = job_title.get_attribute('innerHTML').replace('\n','').strip()
                    company = job.find_element(By.XPATH, ".//a[@data-type='company']")
                    company = company.get_attribute('innerHTML').replace('\n', '').strip()
                    descs.append(job_title)
                    descs.append(link)
                    descs.append(company)
                except:
                    continue

                try:
                    posted_ago = job.find_element(By.XPATH, ".//span[@data-automation='jobListingDate']")
                    posted_ago = posted_ago.get_attribute('innerHTML').replace('\n','').strip()
                    descs.append(posted_ago)
                except:
                    descs.append('')
                
                if len(jobs_scraped) == 0:
                    jobs_scraped = np.append(jobs_scraped, descs)
                else:
                    jobs_scraped = np.vstack([jobs_scraped, descs])
                    
            # limit to max len_jobs only
            if len(jobs_scraped) >= len_jobs:
                jobs_scraped = jobs_scraped[:len_jobs]
                break
                
            next_button = driver.find_element(By.XPATH, ".//a[@aria-label='Next']")
            next_button.click()
        except:
            break
    
    if jobs_scraped.shape == (4,):
        jobs_scraped = np.array([list(jobs_scraped)])
    
    # individual job scraping
    job_descs = np.array([])
    for job in jobs_scraped:
        try:
            driver.get(job[1])
            descs = [job[1]]
            try:
                location = driver.find_element(By.XPATH, "//span[@data-automation='job-detail-location']/a")
                location = location.get_attribute('innerHTML').replace('\n','').strip()
                descs.append(location)
            except:
                descs.append('')
            try:
                job_func = driver.find_element(By.XPATH, "//span[@data-automation='job-detail-classifications']/a")
                job_func = job_func.get_attribute('innerHTML').replace('\n','').strip()
                descs.append(job_func)
            except:
                descs.append('')
            try:
                emp_type = driver.find_element(By.XPATH, "//span[@data-automation='job-detail-work-type']/a")
                emp_type = emp_type.get_attribute('innerHTML').replace('\n','').strip()
                descs.append(emp_type)
            except:
                descs.append('')
            try:
                job_desc = driver.find_element(By.XPATH, "//div[@data-automation='jobAdDetails']/div")
                job_desc = job_desc.get_attribute('innerHTML').replace('\n','').strip()
                descs.append(job_desc)
            except:
                descs.append('')
        except:
            descs = [job[2], '', '', '', '']
        if len(job_descs) == 0:
            job_descs = np.append(job_descs, descs)
        else:
            job_descs = np.vstack([job_descs, descs])
    
    # merging
    jobs_df = pd.DataFrame(jobs_scraped)
    jobs_df.columns = ['title', 'link', 'company', 'posted']
    job_descs_df = pd.DataFrame(job_descs)
    
    # if there is only 1 scraped job
    if job_descs_df.shape == (5,1):
        job_descs_df = job_descs_df.T
    
    job_descs_df.columns = ['link', 'location', 'job_func', 'emp_type', 'job_desc']
    jobstreet_df = jobs_df.merge(job_descs_df, on='link', how='left')
except:
    print("Unable to Scrape Jobstreet")
finally:
    # close the driver
    try:
        driver.close()
    except:
        print("Unable to Scrape Jobstreet")
end = time.time()
print("TOTAL TIME", end-start)

TOTAL TIME 52.150325775146484


# Scraping Kalibrr

In [121]:
job_query = "python-developer"
len_jobs = 30

In [122]:
start = time.time()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = f"https://www.kalibrr.com/home/te/{job_query}"

try:
    driver.get(url)
    job_list = []
    # load job list 
    while len(job_list) < len_jobs:
        # click button to load more jobs until job_list > len_jobs
        try:
            # Re-evaluate job_list after each load
            wait = WebDriverWait(driver, 5)
            job_list = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='k-font-dm-sans k-rounded-lg k-bg-white k-border-solid k-border hover:k-border-2 hover:k-border-primary-color k-border k-group k-flex k-flex-col k-justify-between css-1otdiuc']")))
            
            load_more = wait.until(EC.presence_of_element_located((By.XPATH, "//button[@class='k-btn-primary']")))
            load_more.click()
        except:
            break
    job_list[:len_jobs]
    
    # job catalog scraping
    jobs_scraped = np.array([])
    i = 0
    for job in job_list:
        i += 1
        descs = []
        # scraping job title, company, emp_type, location
        try:
            job_title = job.find_element(By.XPATH, ".//h2[@data-tooltip-id='job-title-tooltip-[object Object]']/a")
            url = job_title.get_attribute('href')
            job_title = job_title.get_attribute('innerHTML').replace('\n', '').strip()
            descs.append(job_title)
            descs.append(url)
        except:
            continue
    
        try:
            company = job.find_element(By.XPATH, ".//span[@class='k-inline-flex k-items-center k-mb-1']/a")
            company = company.text.replace('\n', '').strip()
            descs.append(company)
        except:
            descs.append('')
            
        try:
            emp_type = job.find_element(By.XPATH, "./div[@class='k-relative']/div/span/span[@class='k-text-gray-500']")
            emp_type = emp_type.get_attribute('innerHTML').replace('\n', '').strip()
            descs.append(emp_type)
        except:
            descs.append('')
    
        try:
            location = job.find_element(By.XPATH, "./div[@class='k-relative']/div/span/span[@class='k-text-gray-500 k-block k-pointer-events-none']")
            location = location.get_attribute('innerHTML').replace('\n', '').strip()
            descs.append(location)
        except:
            descs.append('')
            
        if len(jobs_scraped) == 0:
            jobs_scraped = np.append(jobs_scraped, descs)
        else:
            jobs_scraped = np.vstack([jobs_scraped, descs])

        # limit to len_jobs only
        if len(jobs_scraped) >= len_jobs:
            break
    
    if jobs_scraped.shape == (5,):
        jobs_scraped = np.array([list(jobs_scraped)])
    
    # individual job scraping
    job_descs = np.array([])
    for job in jobs_scraped:
        descs = [job[1]]
        try:
            driver.get(job[1])
            # scrape job_func, posted
            try:
                wait = WebDriverWait(driver, timeout=2)
                job_func = wait.until(EC.presence_of_element_located((By.XPATH, ".//div[@class='md:k-flex']//dt[contains(text(),'Job Category')]/following-sibling::dd/a")))
                job_func = job_func.get_attribute('innerHTML').replace('\n', '').strip()
                descs.append(job_func)
            except:
                descs.append('')
            try:
                posted = driver.find_element(By.XPATH, ".//div[@class='k-text-subdued k-text-caption md:k-text-right md:k-absolute md:k-right-0 md:k-top-0 md:k-p-4']/p")
                posted = posted.get_attribute('innerHTML').replace('\n', '').strip()
                descs.append(posted)
            except:
                descs.append('')
            # scrape desc
            job_desc = ''
            try:
                job_desc = driver.find_element(By.XPATH, ".//div[@itemprop='description']")
                job_desc = job_desc.get_attribute('innerHTML')
                job_desc += job_desc
            except:
                job_desc = job_desc
            try:
                job_qual = driver.find_element(By.XPATH, ".//div[@itemprop='qualifications']")
                job_qual = job_qual.get_attribute('innerHTML')
                job_desc += job_qual
            except:
                job_desc = job_desc
            try:
                job_benef = driver.find_element(By.XPATH, ".//div[@itemprop='jobBenefits']")
                job_benef = job_benef.get_attribute('innerHTML').replace('\n', '').strip()
                job_desc += job_qual
            except:
                job_desc = job_desc
            try:
                job_skills = driver.find_element(By.XPATH, ".//ul")
                job_skills = job_skills.get_attribute('innerHTML')
                job_desc += job_skills
            except:
                job_desc = job_desc
        
            descs.append(job_desc)
        
            if len(job_descs) == 0:
                job_descs = np.append(job_descs, descs)
            else:
                job_descs = np.vstack([job_descs, descs])
        except:
            continue
    
    jobs_df = pd.DataFrame(jobs_scraped)
    jobs_df.columns = ['title', 'link' ,'company', 'emp_type', 'location']
    
    job_descs_df = pd.DataFrame(job_descs)
    
    if job_descs_df.shape == (3, 1):
        job_descs_df = job_descs_df.T
    job_descs_df.columns = ['link', 'job_func', 'posted', 'job_desc']
    kalibrr_df = jobs_df.merge(job_descs_df, on='link', how='left')

except:
    print("Unable to Scrape Kalibrr")
finally:
    # close driver
    try:
        driver.close()
    except:
        print("Unable to Scrape Kalibrr")

end = time.time()
print("TOTAL TIME", end-start)    

TOTAL TIME 37.50219678878784


# Checkpoint

In [123]:
# saving
linkedin_df.to_csv('linkedin.csv')
foundit_df.to_csv('foundit.csv')
jobstreet_df.to_csv('jobstreet.csv')
kalibrr_df.to_csv('kalibrr.csv')

In [124]:
# loading
linkedin_df = pd.read_csv('linkedin.csv')
foundit_df = pd.read_csv('foundit.csv')
jobstreet_df = pd.read_csv('jobstreet.csv')
kalibrr_df = pd.read_csv('kalibrr.csv')

# Preprocessing

In [125]:
linkedin_df.columns

Index(['Unnamed: 0', 'title', 'link', 'location', 'company', 'emp_type',
       'job_func', 'job_desc', 'posted'],
      dtype='object')

In [126]:
foundit_df.columns

Index(['Unnamed: 0', 'title', 'company', 'link', 'location', 'posted',
       'emp_type', 'job_func', 'job_desc'],
      dtype='object')

In [127]:
jobstreet_df.columns

Index(['Unnamed: 0', 'title', 'link', 'company', 'posted', 'location',
       'job_func', 'emp_type', 'job_desc'],
      dtype='object')

In [128]:
kalibrr_df.columns

Index(['Unnamed: 0', 'title', 'link', 'company', 'emp_type', 'location',
       'job_func', 'posted', 'job_desc'],
      dtype='object')

In [129]:
# Drop 'Unnamed: 0' column
linkedin_df = linkedin_df.drop(columns=['Unnamed: 0'], errors='ignore')
foundit_df = foundit_df.drop(columns=['Unnamed: 0'], errors='ignore')
jobstreet_df = jobstreet_df.drop(columns=['Unnamed: 0'], errors='ignore')
kalibrr_df = kalibrr_df.drop(columns=['Unnamed: 0'], errors='ignore')

## Standardizing Job Functions

In [130]:
sheet_id = "1FJgg2JWrKfzyWbi-76vKcR1dm-q6RwCAipobzotr_Eg"
sheet_name = "Job_Functions"

url= f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df_jobs = pd.read_csv(url)

df_jobs

,Standardized,Linkedin,Foundit,Kalibrr,Jobstreet
0,Accounting & Auditing,Accounting/Auditing,NaN,Accounting and Finance,Accounting
1,Administration & Office Support,Administrative,Admin/secretarial/front office,Adminstration and Coordination,Administration & Office Support
2,Advertising,NaN,Advertising/entertainment/media,Media and Creatives,"Advertising, Arts & Media"
3,Analyst,Analyst,Analytics/business intelligence,NaN,NaN
4,Architecture,NaN,Architecture/interior design,Architecture and Engineering,Design & Architecture
...,...,...,...,...,...
64,Sales,NaN,Sales/business development,NaN,NaN
65,Sales,Entrepreneurship,Retail chains,NaN,NaN
66,Sales,NaN,Fashion/apparels,NaN,NaN
67,Sciences,Research,NaN,Sciences,Science & Technology


In [131]:
def standardize_job_function_column(df, mapping_dict, source_col):
    df['job_func_stand'] = df[source_col].apply(
        lambda x: ", ".join(
            mapping_dict.get(f.strip(), f.strip()) 
            for f in str(x).split(",") if f.strip()
        )
    )

### Linkedin

In [132]:
linkedin_stand_jf = dict(zip(
    df_jobs.dropna(subset=['Linkedin', 'Standardized'])['Linkedin'],
    df_jobs.dropna(subset=['Linkedin', 'Standardized'])['Standardized']
))

dict(list(linkedin_stand_jf.items())[:5])


{'Accounting/Auditing': 'Accounting & Auditing',
 'Administrative': 'Administration & Office Support',
 'Analyst': 'Analyst',
 'Arts and Design': 'Arts and Graphics Design',
 'Finance': 'Banking & Financial Services'}

In [133]:
standardize_job_function_column(linkedin_df, linkedin_stand_jf, 'job_func')

linkedin_df[['job_func', 'job_func_stand']].head()

,job_func,job_func_stand
0,"Engineering, Information Technology","Engineering, Information Technology"
1,"Engineering, Information Technology","Engineering, Information Technology"
2,"Engineering, Information Technology","Engineering, Information Technology"
3,"Engineering, Information Technology","Engineering, Information Technology"
4,"Engineering, Information Technology","Engineering, Information Technology"


### Foundit

In [134]:
foundit_stand_jf = dict(zip(
    df_jobs.dropna(subset=['Foundit', 'Standardized'])['Foundit'],
    df_jobs.dropna(subset=['Foundit', 'Standardized'])['Standardized']
))


dict(list(foundit_stand_jf.items())[:5])


{'Admin/secretarial/front office': 'Administration & Office Support',
 'Advertising/entertainment/media': 'Advertising',
 'Analytics/business intelligence': 'Analyst',
 'Architecture/interior design': 'Architecture',
 'Arts/creative/graphics design': 'Arts and Graphics Design'}

In [135]:
# standardize_job_function_column(foundit_df, foundit_stand_jf, 'job_func')

# foundit_df[['job_func', 'job_func_stand']]

In [136]:
# foundit_df[['job_func', 'job_func_stand']]

### Kalibrr

In [137]:
kalibrr_stand_jf = dict(zip(
    df_jobs.dropna(subset=['Kalibrr', 'Standardized'])['Kalibrr'],
    df_jobs.dropna(subset=['Kalibrr', 'Standardized'])['Standardized']
))


dict(list(kalibrr_stand_jf.items())[:5])

{'Accounting and Finance': 'Accounting & Auditing',
 'Adminstration and Coordination': 'Administration & Office Support',
 'Media and Creatives': 'Advertising',
 'Architecture and Engineering': 'Architecture',
 'Arts and Sports': 'Arts and Graphics Design'}

In [138]:
standardize_job_function_column(kalibrr_df, kalibrr_stand_jf, 'job_func')

kalibrr_df[['job_func', 'job_func_stand']].head()

,job_func,job_func_stand
0,IT and Software,Information Technology
1,IT and Software,Information Technology
2,IT and Software,Information Technology
3,IT and Software,Information Technology
4,IT and Software,Information Technology


### Jobstreet

In [139]:
jobstreet_stand_jf = dict(zip(
    df_jobs.dropna(subset=['Jobstreet', 'Standardized'])['Jobstreet'],
    df_jobs.dropna(subset=['Jobstreet', 'Standardized'])['Standardized']
))

dict(list(jobstreet_stand_jf.items())[:5])

{'Accounting': 'Accounting & Auditing',
 'Administration & Office Support': 'Administration & Office Support',
 'Advertising, Arts & Media': 'Advertising',
 'Design & Architecture': 'Architecture',
 'Banking & Financial Services': 'Banking & Financial Services'}

In [140]:
jobstreet_df['job_func_clean'] = jobstreet_df['job_func'].str.extract(r'\(([^)]+)\)')
jobstreet_df['job_func_clean'] = jobstreet_df['job_func_clean'].str.replace('&amp;', '&', regex=False)


jobstreet_df['job_func_clean'].head()

0    Information & Communication Technology
1    Information & Communication Technology
2    Information & Communication Technology
3    Information & Communication Technology
4    Information & Communication Technology
Name: job_func_clean, dtype: object

In [141]:
standardize_job_function_column(jobstreet_df, jobstreet_stand_jf, 'job_func_clean')

jobstreet_df[['job_func_clean', 'job_func_stand']].head()

,job_func_clean,job_func_stand
0,Information & Communication Technology,Information Technology
1,Information & Communication Technology,Information Technology
2,Information & Communication Technology,Information Technology
3,Information & Communication Technology,Information Technology
4,Information & Communication Technology,Information Technology


## Standardizing Location

In [142]:
sheet_id = "1FJgg2JWrKfzyWbi-76vKcR1dm-q6RwCAipobzotr_Eg"
sheet_name = "Location"

url= f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df_locations = pd.read_csv(url)

df_locations['city_clean'] = df_locations['City/Province'].str.replace(r'\s+City$', '', regex=True)


df_locations

,City/Province,Region,Type,city_clean
0,Caloocan City,National Capital Region (NCR),City,Caloocan
1,Las Pinas City,National Capital Region (NCR),City,Las Pinas
2,Makati City,National Capital Region (NCR),City,Makati
3,Malabon City,National Capital Region (NCR),City,Malabon
4,Mandaluyong City,National Capital Region (NCR),City,Mandaluyong
...,...,...,...,...
225,Lanao del Sur,Bangsamoro Autonomous Region in Muslim Mindana...,Province,Lanao del Sur
226,Sulu,Bangsamoro Autonomous Region in Muslim Mindana...,Province,Sulu
227,Tawi-Tawi,Bangsamoro Autonomous Region in Muslim Mindana...,Province,Tawi-Tawi
228,Maguindanao del Norte,Bangsamoro Autonomous Region in Muslim Mindana...,Province,Maguindanao del Norte


In [143]:
def normalize(text):
    text = str(text).lower().replace(',', '').replace('-', '').replace('philippines', '').strip()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode() 
    
    return text

def smart_location_match(raw_location, keywords, df_lookup):
    raw = normalize(raw_location)

    if raw_location.lower().strip() == "philippines" or raw == "other":
        return "Philippines"

    # Exact keyword match, use fuzzy otherwise
    for keyword in keywords:
        if normalize(keyword) in raw:
            match = keyword
            break
    else:
        match, score, _ = process.extractOne(raw, keywords, processor=normalize)
        if score <= 80:
            return "Other"
    
    if match in df_lookup['city_clean'].values:
        matches = df_lookup[df_lookup['city_clean'] == match]
        region = matches['Region'].values[0]
        type_ = matches['Type'].values[0]

        if type_ == 'City':
            return f"{match} City, {region}"
        else:
            return f"{match}, {region}"
        
    elif match in df_lookup['Region'].values:
        return match
    else:
        return "Other"

keywords = pd.concat([
    df_locations['city_clean'], 
    df_locations['Region']
]).dropna().unique().tolist()

### Linkedin

In [144]:
linkedin_df['location_standardized'] = linkedin_df['location'].apply(lambda x: smart_location_match(x, keywords, df_locations))

linkedin_df['location_standardized'].head(10)

0    Manila City, National Capital Region (NCR)
1                                   Philippines
2    Manila City, National Capital Region (NCR)
3    Quezon City, National Capital Region (NCR)
4    Manila City, National Capital Region (NCR)
5    Manila City, National Capital Region (NCR)
6    Manila City, National Capital Region (NCR)
7    Manila City, National Capital Region (NCR)
8    Manila City, National Capital Region (NCR)
9    Manila City, National Capital Region (NCR)
Name: location_standardized, dtype: object

In [145]:
linkedin_df['location_standardized'].value_counts()

location_standardized
Manila City, National Capital Region (NCR)    16
Philippines                                    8
Quezon City, National Capital Region (NCR)     1
National Capital Region (NCR)                  1
Taguig City, National Capital Region (NCR)     1
Makati City, National Capital Region (NCR)     1
Pasig City, National Capital Region (NCR)      1
Cebu City, Central Visayas (Region VII)        1
Name: count, dtype: int64

### Foundit

In [146]:
foundit_df['location_standardized'] = foundit_df['location'].apply(lambda x: smart_location_match(x, keywords, df_locations))

foundit_df['location_standardized'].head(10)

0    Quezon City, National Capital Region (NCR)
1                  Central Visayas (Region VII)
2                                   Philippines
3                                   Philippines
4    Manila City, National Capital Region (NCR)
5    Manila City, National Capital Region (NCR)
6                                   Philippines
7                                   Philippines
8                                   Philippines
9                                   Philippines
Name: location_standardized, dtype: object

In [147]:
foundit_df['location_standardized'].value_counts()

location_standardized
Philippines                                   13
Taguig City, National Capital Region (NCR)     6
Manila City, National Capital Region (NCR)     5
Quezon City, National Capital Region (NCR)     4
Makati City, National Capital Region (NCR)     2
Central Visayas (Region VII)                   1
Name: count, dtype: int64

### Kalibrr

In [148]:
kalibrr_df['location_standardized'] = kalibrr_df['location'].apply(lambda x: smart_location_match(x, keywords, df_locations))

kalibrr_df['location_standardized'].head(10)

0                                              Other
1      Paranaque City, National Capital Region (NCR)
2         Makati City, National Capital Region (NCR)
3    Mandaluyong City, National Capital Region (NCR)
4          Pasig City, National Capital Region (NCR)
5         Manila City, National Capital Region (NCR)
6         Manila City, National Capital Region (NCR)
7         Quezon City, National Capital Region (NCR)
8      Paranaque City, National Capital Region (NCR)
9    Mandaluyong City, National Capital Region (NCR)
Name: location_standardized, dtype: object

In [149]:
kalibrr_df['location_standardized'].value_counts()

location_standardized
Manila City, National Capital Region (NCR)         9
Quezon City, National Capital Region (NCR)         5
Makati City, National Capital Region (NCR)         4
Pasig City, National Capital Region (NCR)          4
Paranaque City, National Capital Region (NCR)      3
Mandaluyong City, National Capital Region (NCR)    3
Other                                              1
Taguig City, National Capital Region (NCR)         1
Name: count, dtype: int64

### Jobstreet

In [150]:
jobstreet_df['location_standardized'] = jobstreet_df['location'].apply(lambda x: smart_location_match(x, keywords, df_locations))

jobstreet_df['location_standardized'].head(10)

0    Manila City, National Capital Region (NCR)
1    Manila City, National Capital Region (NCR)
2    Manila City, National Capital Region (NCR)
3    Manila City, National Capital Region (NCR)
4    Manila City, National Capital Region (NCR)
5    Manila City, National Capital Region (NCR)
6    Manila City, National Capital Region (NCR)
7    Manila City, National Capital Region (NCR)
8    Makati City, National Capital Region (NCR)
9               Rizal, CALABARZON (Region IV-A)
Name: location_standardized, dtype: object

In [151]:
jobstreet_df['location_standardized'].value_counts()

location_standardized
Manila City, National Capital Region (NCR)         24
Makati City, National Capital Region (NCR)          2
Mandaluyong City, National Capital Region (NCR)     2
Rizal, CALABARZON (Region IV-A)                     1
Laguna, CALABARZON (Region IV-A)                    1
Name: count, dtype: int64

In [152]:
linkedin_df_stand = linkedin_df.drop(['job_func','location'], axis=1)
kalibrr_df_stand = kalibrr_df.drop(['job_func','location'], axis=1)
# foundit_df_stand = foundit_df.drop(['job_func','location'], axis=1)
foundit_df_stand = foundit_df.drop('location', axis=1)
jobstreet_df_stand = jobstreet_df.drop(['job_func','job_func_clean','location'], axis=1)

## Standardizing Job Type

In [153]:
sheet_id = "1FJgg2JWrKfzyWbi-76vKcR1dm-q6RwCAipobzotr_Eg"
sheet_name = "Job_Type"

url= f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df_job_type = pd.read_csv(url)

df_job_type

,Standardized,Linkedin,Foundit,Kalibrr,Jobstreet
0,Full-time,Full-time,Permanent Job,Full time,Full time
1,Contract/Temporary,Contract,Contract Job,Contractual,Contract/Temp
2,Internship,Internship,NaN,NaN,NaN
3,Part-time,Part-time,NaN,Part time,Part time
4,Contract/Temporary,Temporary,NaN,Freelance,Casual/Vacation


In [154]:
def standardize_job_type(df, mapping_dict, source_col):
    df['job_type_stand'] = df[source_col].apply(
        lambda x: ", ".join(
            mapping_dict.get(f.strip(), f.strip()) 
            for f in str(x).split(",") if f.strip()
        )
    )

### Linkedin

In [155]:
linkedin_df['emp_type'].value_counts()

emp_type
Full-time    28
Contract      1
Name: count, dtype: int64

In [156]:
# Remove 'Other' -> job type is multi-select for this site so retain the other job type
# 'Work From Home' and 'Jobs for Women' -> for foundit

def clean_job_type(val):
    job_types = [t.strip() for t in str(val).split(',')]
    
    remove = {'Other','Work From Home', 'Jobs for Women'}
    
    cleaned = [x for x in job_types if x not in remove]
    
    return ", ".join(cleaned) if cleaned else None

linkedin_df['emp_type'] = linkedin_df['emp_type'].apply(clean_job_type)


In [157]:
linkedin_stand_type = dict(zip(
    df_job_type.dropna(subset=['Linkedin', 'Standardized'])['Linkedin'],
    df_job_type.dropna(subset=['Linkedin', 'Standardized'])['Standardized']
))


dict(list(linkedin_stand_type.items())[:5])

{'Full-time': 'Full-time',
 'Contract': 'Contract/Temporary',
 'Internship': 'Internship',
 'Part-time': 'Part-time',
 'Temporary': 'Contract/Temporary'}

In [158]:
standardize_job_type(linkedin_df, linkedin_stand_type, 'emp_type')

linkedin_df[['emp_type', 'job_type_stand']].head()

,emp_type,job_type_stand
0,Full-time,Full-time
1,Full-time,Full-time
2,Full-time,Full-time
3,Full-time,Full-time
4,Full-time,Full-time


In [159]:
linkedin_df['job_type_stand'].value_counts()

job_type_stand
Full-time             28
Contract/Temporary     1
nan                    1
Name: count, dtype: int64

### Foundit

In [160]:
foundit_df['emp_type'].value_counts()

emp_type
Permanent Job    31
Name: count, dtype: int64

In [161]:
# Remove 'Work From Home' and 'Jobs for Women' -> job type is multi-select for this site so retain the other job type
foundit_df['emp_type'] = foundit_df['emp_type'].apply(clean_job_type)


In [162]:
# Internship and part-time job type are inferred from job title
foundit_df.loc[foundit_df['title'].str.contains(r'\bintern\b|\binternship\b', case=False, na=False),'job_type_stand'] = 'Internship'
foundit_df.loc[foundit_df['title'].str.contains(r'\bpart-time\b|\bpart time\b', case=False, na=False),'job_type_stand'] = 'Part-time'

In [163]:
foundit_stand_type = dict(zip(
    df_job_type.dropna(subset=['Foundit', 'Standardized'])['Foundit'],
    df_job_type.dropna(subset=['Foundit', 'Standardized'])['Standardized']
))

standardize_job_type(foundit_df, foundit_stand_type, 'emp_type')

foundit_df[['emp_type', 'job_type_stand']].value_counts()

emp_type       job_type_stand
Permanent Job  Full-time         31
Name: count, dtype: int64

In [164]:
foundit_df['job_type_stand'].value_counts()

job_type_stand
Full-time    31
Name: count, dtype: int64

### Kalibrr

In [165]:
kalibrr_df['emp_type'].value_counts()

emp_type
Full time    29
Freelance     1
Name: count, dtype: int64

In [166]:
# Internship job type is inferred from job title
kalibrr_df.loc[kalibrr_df['title'].str.contains(r'\bintern\b|\binternship\b', case=False, na=False),'job_type_stand'] = 'Internship'

In [167]:
kalibrr_stand_type = dict(zip(
    df_job_type.dropna(subset=['Kalibrr', 'Standardized'])['Kalibrr'],
    df_job_type.dropna(subset=['Kalibrr', 'Standardized'])['Standardized']
))

standardize_job_type(kalibrr_df, kalibrr_stand_type, 'emp_type')

kalibrr_df[['emp_type', 'job_type_stand']].value_counts()


emp_type   job_type_stand    
Full time  Full-time             29
Freelance  Contract/Temporary     1
Name: count, dtype: int64

In [168]:
kalibrr_df['job_type_stand'].value_counts()

job_type_stand
Full-time             29
Contract/Temporary     1
Name: count, dtype: int64

### Jobstreet

In [169]:
jobstreet_df['emp_type'].value_counts()

emp_type
Full time    30
Name: count, dtype: int64

In [170]:
# Internship job type is inferred from job title
jobstreet_df.loc[jobstreet_df['title'].str.contains(r'\bintern\b|\binternship\b', case=False, na=False),'job_type_stand'] = 'Internship'

In [171]:
jobstreet_stand_type = dict(zip(
    df_job_type.dropna(subset=['Jobstreet', 'Standardized'])['Jobstreet'],
    df_job_type.dropna(subset=['Jobstreet', 'Standardized'])['Standardized']
))

standardize_job_type(jobstreet_df, jobstreet_stand_type, 'emp_type')

jobstreet_df[['emp_type', 'job_type_stand']].value_counts()


emp_type   job_type_stand
Full time  Full-time         30
Name: count, dtype: int64

In [172]:
jobstreet_df['job_type_stand'].value_counts()

job_type_stand
Full-time    30
Name: count, dtype: int64

In [173]:
linkedin_df_stand = linkedin_df.drop(['job_func','location','emp_type'], axis=1)
kalibrr_df_stand = kalibrr_df.drop(['job_func','location','emp_type'], axis=1)
# foundit_df_stand = foundit_df.drop(['job_func','location'], axis=1)
foundit_df_stand = foundit_df.drop(['location','emp_type'], axis=1)
jobstreet_df_stand = jobstreet_df.drop(['job_func','job_func_clean','location','emp_type'], axis=1)

## Merging the Job Listings

In [174]:
linkedin_df_stand.columns

Index(['title', 'link', 'company', 'job_desc', 'posted', 'job_func_stand',
       'location_standardized', 'job_type_stand'],
      dtype='object')

In [175]:
kalibrr_df_stand.columns

Index(['title', 'link', 'company', 'posted', 'job_desc', 'job_func_stand',
       'location_standardized', 'job_type_stand'],
      dtype='object')

In [176]:
foundit_df_stand.columns

Index(['title', 'company', 'link', 'posted', 'job_func', 'job_desc',
       'location_standardized', 'job_type_stand'],
      dtype='object')

In [177]:
jobstreet_df_stand.columns

Index(['title', 'link', 'company', 'posted', 'job_desc', 'job_func_stand',
       'location_standardized', 'job_type_stand'],
      dtype='object')

In [178]:
# merge the four dataframes

columns = ['title', 'company', 'link', 'job_desc','job_func_stand', 'location_standardized', 'job_type_stand']

linkedin_df_stand['source'] = 'Linkedin'
kalibrr_df_stand['source'] = 'Kalibrr'
foundit_df_stand['source'] = 'Foundit'
jobstreet_df_stand['source'] = 'Jobstreet'

for df in [linkedin_df_stand, kalibrr_df_stand, foundit_df_stand, jobstreet_df_stand]:
    for col in columns:
        if col not in df.columns:
            df[col] = np.nan


job_listings = pd.concat([
    linkedin_df_stand[columns + ['source']],
    kalibrr_df_stand[columns + ['source']],
    foundit_df_stand[columns + ['source']],
    jobstreet_df_stand[columns + ['source']]
], ignore_index=True)

job_listings

,title,company,link,job_desc,job_func_stand,location_standardized,job_type_stand,source
0,Junior Web Developer,INQUIRER.net,https://ph.linkedin.com/jobs/view/junior-web-d...,\n <strong>Key Responsibilities<br><b...,"Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin
1,Junior Python Developer,ShipERP,https://ph.linkedin.com/jobs/view/junior-pytho...,\n <strong>Position Overview<br><br><...,"Engineering, Information Technology",Philippines,Full-time,Linkedin
2,Python Developer,Theoria Medical,https://ph.linkedin.com/jobs/view/python-devel...,\n <p><strong>Position Type</strong>:...,"Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin
3,Junior Back End Developer,Media Meter Inc.,https://ph.linkedin.com/jobs/view/junior-back-...,\n <strong>Is this role right for you...,"Engineering, Information Technology","Quezon City, National Capital Region (NCR)",Full-time,Linkedin
4,Python Developer,Theoria Medical,https://ph.linkedin.com/jobs/view/python-devel...,\n <p><strong>Position Type</strong>:...,"Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin
...,...,...,...,...,...,...,...,...
116,R0005686: Fullstack Development Engineer,TREND MICRO INCORPORATED-PHILIPPINE BRANCH,https://ph.jobstreet.com/job/82482187?type=sta...,<p><strong>Overview</strong></p><p>&nbsp;</p><...,Information Technology,"Manila City, National Capital Region (NCR)",Full-time,Jobstreet
117,Data Analyst - Python,Aeon Credit Service Systems (Philippines) Inc.,https://ph.jobstreet.com/job/82535741?type=sta...,<p><strong>Job Brief:</strong></p><p>We are lo...,Sciences,"Mandaluyong City, National Capital Region (NCR)",Full-time,Jobstreet
118,Big Data Engineer,FIRSTMAC OPERATIONS CENTER PTY LTD-PHILIPPINE ...,https://ph.jobstreet.com/job/82683949?type=sta...,<p><strong>POSITION PURPOSE: </strong>The big ...,Information Technology,"Manila City, National Capital Region (NCR)",Full-time,Jobstreet
119,Data Engineer,EY GLOBAL DELIVERY SERVICES (GDS) PHILIPPINES,https://ph.jobstreet.com/job/83097417?type=sta...,<p><strong>The opportunity</strong></p><p>We’r...,Information Technology,"Manila City, National Capital Region (NCR)",Full-time,Jobstreet


## Dropping Duplicates

In [179]:
duplicates = job_listings[job_listings.duplicated(subset=['title', 'company'], keep=False)]
duplicates = duplicates.groupby(['title', 'company','source']).size().reset_index(name='count')
duplicates = duplicates.sort_values(by='count', ascending=False)
duplicates

,title,company,source,count
9,Python Developer,Theoria Medical,Linkedin,15
0,Full Stack Developer (React/Node/Python) - Nig...,MicroSourcing,Foundit,2
3,Full-Stack Developer (Python/Django &amp; React),One Outsource,Foundit,2
10,Python Developer with Data Engineering experie...,ERNI,Linkedin,2
11,Software Developer (Python),"Risewave Consulting, Inc.",Foundit,2
1,Full Stack Python Developer,Tahche Careers,Foundit,1
2,Full Stack Python Developer,Tahche Careers,Linkedin,1
4,Junior Python Developer,ShipERP,Foundit,1
5,Junior Python Developer,ShipERP,Linkedin,1
6,Python Developer,Genzeon,Foundit,1


In [180]:
# retain first listing
job_listings = job_listings.drop_duplicates(subset=['title', 'company'], keep='first')
job_listings


,title,company,link,job_desc,job_func_stand,location_standardized,job_type_stand,source
0,Junior Web Developer,INQUIRER.net,https://ph.linkedin.com/jobs/view/junior-web-d...,\n <strong>Key Responsibilities<br><b...,"Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin
1,Junior Python Developer,ShipERP,https://ph.linkedin.com/jobs/view/junior-pytho...,\n <strong>Position Overview<br><br><...,"Engineering, Information Technology",Philippines,Full-time,Linkedin
2,Python Developer,Theoria Medical,https://ph.linkedin.com/jobs/view/python-devel...,\n <p><strong>Position Type</strong>:...,"Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin
3,Junior Back End Developer,Media Meter Inc.,https://ph.linkedin.com/jobs/view/junior-back-...,\n <strong>Is this role right for you...,"Engineering, Information Technology","Quezon City, National Capital Region (NCR)",Full-time,Linkedin
10,Python Engineer,CI&amp;T,https://ph.linkedin.com/jobs/view/python-engin...,\n <p><span>We are <strong>tech trans...,"Engineering, Information Technology",National Capital Region (NCR),Full-time,Linkedin
...,...,...,...,...,...,...,...,...
116,R0005686: Fullstack Development Engineer,TREND MICRO INCORPORATED-PHILIPPINE BRANCH,https://ph.jobstreet.com/job/82482187?type=sta...,<p><strong>Overview</strong></p><p>&nbsp;</p><...,Information Technology,"Manila City, National Capital Region (NCR)",Full-time,Jobstreet
117,Data Analyst - Python,Aeon Credit Service Systems (Philippines) Inc.,https://ph.jobstreet.com/job/82535741?type=sta...,<p><strong>Job Brief:</strong></p><p>We are lo...,Sciences,"Mandaluyong City, National Capital Region (NCR)",Full-time,Jobstreet
118,Big Data Engineer,FIRSTMAC OPERATIONS CENTER PTY LTD-PHILIPPINE ...,https://ph.jobstreet.com/job/82683949?type=sta...,<p><strong>POSITION PURPOSE: </strong>The big ...,Information Technology,"Manila City, National Capital Region (NCR)",Full-time,Jobstreet
119,Data Engineer,EY GLOBAL DELIVERY SERVICES (GDS) PHILIPPINES,https://ph.jobstreet.com/job/83097417?type=sta...,<p><strong>The opportunity</strong></p><p>We’r...,Information Technology,"Manila City, National Capital Region (NCR)",Full-time,Jobstreet


In [182]:
job_listings.shape

(99, 8)